In [1]:
import pandas as pd
import numpy as np
import math
from scipy.stats import wilcoxon

In [2]:
def regulate_metrics(values, lenght):
    if len(values) != lenght:

        size_of_batch = len(values) / lenght
        if type(size_of_batch) == float:
            size_of_batch = math.ceil(size_of_batch)
            pad = size_of_batch * lenght
            values = np.pad(values, (pad - len(values)))


        new_values = []
        old_i = 0
        for i in range(size_of_batch, len(values), size_of_batch):
            new_values.append(np.sum(values[old_i:i]))
            old_i = i
        

    return new_values

In [9]:
def load_error_and_rank_metrics(algorithms, dataset, emb, lenght, path="results/"):

    labels = ['MSE', 'MAE', 'RMSE', 'PRECISION', 
              'RECALL', 'DIVERSITY', 'NOVELTY', 'NDCG']
    
    alg_name_1 = [""]*len(labels) 
    alg_value_1 = [1.0]*len(labels[:3]) + [0.0]*len(labels[3:])

    alg_name_2 = [""]*len(labels)
    alg_value_2 = [1.0]*len(labels[:3]) + [0.0]*len(labels[3:])

    draw = []
    metrics_matrix = []
    
    for algorithm in algorithms:
            error_metrics = pd.read_csv(
                path + algorithm + "_" + dataset + "_" + emb + "_results_error.csv")
            
            rank_metrics = pd.read_csv(
                path + algorithm + "_" + dataset + "_" + emb + "_results_rank.csv")
            

            values = np.concatenate([error_metrics.mean().values, 
                                    rank_metrics.mean().values])
            
            for i, m in enumerate(values[:3]):
                if m < alg_value_1[i]:
                    alg_name_1[i] = algorithm
                    alg_value_1[i] = m

                elif m < alg_value_2[i]:
                    alg_name_2[i] = algorithm
                    alg_value_2[i] = m

            for i, m in enumerate(values[3:]):
                idx = i + 3
                if m > alg_value_1[idx]:
                    alg_name_1[idx] = algorithm
                    alg_value_1[idx] = m

                elif m > alg_value_2[idx]:
                    alg_name_2[idx] = algorithm
                    alg_value_2[idx] = m


            metrics_matrix.append(values)

    # for i, (alg1, alg2) in enumerate(zip(alg_name_1, alg_name_2)):
        
    #     alg1_metrics_error = pd.read_csv(
    #             path + alg1 + "_" + dataset + "_" + emb + "_results_error.csv").values.transpose()
            
    #     alg1_metrics_rank = pd.read_csv(
    #             path + alg1 + "_" + dataset + "_" + emb + "_results_rank.csv").values.transpose()
        
    #     alg2_metrics_error = pd.read_csv(
    #             path + alg2 + "_" + dataset + "_" + emb + "_results_error.csv").values.transpose()
            
    #     alg2_metrics_rank = pd.read_csv(
    #             path + alg2 + "_" + dataset + "_" + emb + "_results_rank.csv").values.transpose()
        
    #     if i < 7:
    #         alg1_metric = alg1_metrics_error[i]
    #         alg2_metric = alg2_metrics_error[i]
    #     else:
    #         alg1_metric = alg1_metrics_rank[0]
    #         alg2_metric = alg2_metrics_rank[0]

    #     if len(alg1_metric) > lenght:
    #         alg1_metric = regulate_metrics(alg1_metric, lenght)
    #     if len(alg2_metric) > lenght:
    #         alg2_metric = regulate_metrics(alg2_metric, lenght)

    #     if not (np.array(alg1_metric) - np.array(alg2_metric)).any():
    #         test = 0.0
    #     else:
    #         test = wilcoxon(alg1_metric, alg2_metric).pvalue
            
    #     if test < 0.05:
    #         draw.append('Draw')
    #     else:
    #         draw.append('Winner')


    df = pd.DataFrame(np.array(metrics_matrix),
                      columns=labels, index=algorithms)
    
    # rank = pd.DataFrame({"Metric": labels,
    #                     "Winner":alg_name_1,
    #                      "2 Place": alg_name_2,
    #                      "Draw": draw})
    rank = []

    return df.round(3), rank
    
    # return  error_metrics.mean(), rank_metrics.mean()

In [154]:
algorithms = ["DeepCoNN",
              "ConvMF", "TARMF", "CARM"]


df, draw = load_error_and_rank_metrics(algorithms, "AMAZON_FASHION_data", "word2vec", 3)

/home/guilherme/iRev/.venv/lib/python3.10/site-packages/scipy/stats/_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [5]:
algorithms = ["DeepCoNN", "ConvMF", "NARRE", "D_ATTN", "MPCN",
            "ALFM", "TRANSNET", "A3NCF", "TARMF", "ANR",
            "CARL", "DAML", "CARP", "CARM"]


df, draw = load_error_and_rank_metrics(algorithms, "Digital_Music_data", "word2vec", 133)
df

/home/guilherme/iRev/.venv/lib/python3.10/site-packages/scipy/stats/_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,MSE,MAE,RMSE,PRECISION,RECALL,DIVERSITY,NOVELTY,NDCG
DeepCoNN,0.347,0.384,0.583,0.463,0.319,0.356,0.061,0.000
ConvMF,0.411,0.452,0.634,0.525,0.274,0.354,0.071,0.000
NARRE,0.321,0.345,0.559,0.497,0.345,0.362,0.043,0.016
D_ATTN,0.347,0.347,0.583,0.481,0.304,0.369,0.043,0.024
MPCN,0.405,0.421,0.630,0.525,0.261,0.360,0.048,0.040
ALFM,0.493,0.479,0.694,0.799,0.224,0.358,0.053,0.016
TRANSNET,0.493,0.474,0.694,0.799,0.224,0.360,0.053,0.000
A3NCF,0.493,0.472,0.694,0.799,0.224,0.360,0.053,0.024
TARMF,0.376,0.461,0.608,0.460,0.339,0.371,0.155,0.032
ANR,0.379,0.373,0.608,0.489,0.286,0.370,0.049,0.024


In [152]:
draw

,Metric,Winner,2 Place,Draw
0,MSE,CARP,HRDR,Draw
1,MAE,CARP,HRDR,Draw
2,RMSE,DAML,CARP,Draw
3,PRECISION,ALFM,TRANSNET,Draw
4,RECALL,DAML,CARP,Draw
5,DIVERSITY,DAML,ANR,Draw
6,NOVELTY,DAML,ALFM,Draw
7,NDCG,MPCN,TARMF,Winner


In [108]:
daml = pd.read_csv("results/DAML_Digital_Music_data_word2vec_results_error.csv")
daml_recall = daml['recall'].values

narre = pd.read_csv("results/NARRE_Digital_Music_data_word2vec_results_error.csv")
narre_recall = narre['recall'].values

In [110]:
wilcoxon(regulate_metrics(daml_recall, 133), narre_recall)

WilcoxonResult(statistic=0.0, pvalue=1.4317759392997278e-23)

In [18]:
pd.read_csv("results/DeepCoNN_Tucso_data_word2vec_results_error.csv").shape

(254, 7)

In [154]:
df, draw = load_error_and_rank_metrics(algorithms, "Musical_Instruments_data", "word2vec", 181)
df

/home/guilherme/iRev/.venv/lib/python3.10/site-packages/scipy/stats/_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,MSE,MAE,RMSE,PRECISION,RECALL,DIVERSITY,NOVELTY,NDCG
DeepCoNN,0.884,0.667,0.935,0.442,0.243,0.206,0.070,0.023
ConvMF,0.927,0.753,0.959,0.507,0.222,0.297,0.119,0.008
NARRE,0.862,0.670,0.924,0.414,0.256,0.240,0.075,0.023
D_ATTN,0.889,0.685,0.939,0.402,0.247,0.237,0.075,0.023
MPCN,0.922,0.718,0.955,0.509,0.234,0.216,0.072,0.023
ALFM,0.841,0.637,0.912,0.535,0.262,0.206,0.069,0.016
TRANSNET,0.977,0.747,0.983,0.164,0.200,0.213,0.074,0.000
A3NCF,0.977,0.746,0.983,0.164,0.200,0.210,0.074,0.000
TARMF,0.882,0.720,0.935,0.386,0.244,0.316,0.134,0.016
ANR,0.903,0.650,0.945,0.497,0.239,0.162,0.106,0.023


In [155]:
draw

,Metric,Winner,2 Place,Draw
0,MSE,ALFM,CARL,Draw
1,MAE,ALFM,ANR,Draw
2,RMSE,DAML,CARL,Draw
3,PRECISION,ALFM,ANR,Draw
4,RECALL,DAML,CARL,Draw
5,DIVERSITY,TARMF,NARRE,Draw
6,NOVELTY,DAML,ANR,Draw
7,NDCG,CARM,NARRE,Winner


In [172]:
algorithms = ["DeepCoNN", "ConvMF", "NARRE", "D_ATTN", "MPCN",
            "ALFM", "TRANSNET", "A3NCF", "TARMF", "ANR",
            "CARL", "DAML", "CARP", "CARM"]

df, draw = load_error_and_rank_metrics(algorithms, "Office_Products_data", "word2vec", 625)
df

/home/guilherme/iRev/.venv/lib/python3.10/site-packages/scipy/stats/_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,MSE,MAE,RMSE,PRECISION,RECALL,DIVERSITY,NOVELTY,NDCG
DeepCoNN,0.933,0.687,0.961,0.399,0.244,0.201,0.075,0.023
ConvMF,0.984,0.731,0.986,0.474,0.226,0.194,0.074,0.023
NARRE,0.922,0.676,0.955,0.379,0.248,0.189,0.070,0.023
D_ATTN,0.928,0.664,0.958,0.385,0.243,0.163,0.071,0.016
MPCN,0.970,0.706,0.979,0.458,0.231,0.175,0.071,0.008
ALFM,1.028,0.754,1.008,0.152,0.202,0.181,0.073,0.016
TRANSNET,1.028,0.754,1.008,0.152,0.202,0.181,0.073,0.010
A3NCF,1.028,0.755,1.008,0.152,0.202,0.182,0.073,0.016
TARMF,0.922,0.717,0.956,0.371,0.248,0.275,0.129,0.016
ANR,0.946,0.709,0.967,0.466,0.236,0.201,0.075,0.008


In [173]:
draw

,Metric,Winner,2 Place,Draw
0,MSE,TARMF,CARM,Draw
1,MAE,D_ATTN,CARP,Draw
2,RMSE,DAML,TARMF,Draw
3,PRECISION,CARL,ANR,Draw
4,RECALL,DAML,CARM,Draw
5,DIVERSITY,TARMF,CARM,Draw
6,NOVELTY,DAML,CARM,Draw
7,NDCG,CARM,ConvMF,Winner


In [10]:
df, draw = load_error_and_rank_metrics(algorithms, "Tamp_data", "word2vec", 264)
df

,MSE,MAE,RMSE,PRECISION,RECALL,DIVERSITY,NOVELTY,NDCG
DeepCoNN,1.696,1.008,1.299,0.359,0.276,0.120,0.094,0.016
ConvMF,1.647,1.056,1.281,0.347,0.239,0.045,0.141,0.016
NARRE,1.631,0.988,1.274,0.372,0.276,0.133,0.093,0.008
D_ATTN,1.715,1.038,1.307,0.379,0.280,0.036,0.156,0.016
MPCN,1.791,1.081,1.336,0.322,0.235,0.082,0.107,0.008
ALFM,1.533,0.997,1.236,0.313,0.255,0.103,0.091,0.008
TRANSNET,1.756,1.082,1.323,0.288,0.222,0.120,0.095,0.000
A3NCF,1.962,1.184,1.399,0.230,0.200,0.067,0.120,0.016
TARMF,1.702,1.007,1.302,0.384,0.284,0.085,0.103,0.016
ANR,1.657,1.002,1.284,0.318,0.251,0.192,0.152,0.016


In [12]:
df, _ = load_error_and_rank_metrics(algorithms, "Philladelphi_data", "word2vec", 610)
df

,MSE,MAE,RMSE,PRECISION,RECALL,DIVERSITY,NOVELTY,NDCG
DeepCoNN,1.487,0.952,1.217,0.358,0.261,0.208,0.085,0.008
ConvMF,1.416,0.952,1.188,0.346,0.242,0.197,0.084,0.008
NARRE,1.412,0.923,1.185,0.365,0.267,0.225,0.084,0.000
D_ATTN,1.473,0.941,1.211,0.364,0.254,0.250,0.097,0.008
MPCN,1.586,0.995,1.256,0.322,0.225,0.224,0.092,0.008
ALFM,1.372,0.926,1.169,0.400,0.255,0.238,0.089,0.008
TRANSNET,1.711,1.076,1.306,0.280,0.200,0.173,0.098,0.000
A3NCF,1.711,1.075,1.306,0.280,0.200,0.175,0.098,0.000
TARMF,1.479,0.932,1.213,0.377,0.272,0.240,0.091,0.000
ANR,1.450,0.932,1.201,0.340,0.249,0.300,0.137,0.000


In [21]:
df, _ = load_error_and_rank_metrics(algorithms, "Tucso_data", "word2vec", 254)
df = df.fillna(0.006).round(3)
df['DIVERSITY'] = df['DIVERSITY'] * -1

In [22]:
df

,MSE,MAE,RMSE,PRECISION,RECALL,DIVERSITY,NOVELTY,NDCG
DeepCoNN,1.849,1.078,1.357,0.349,0.272,0.046,0.102,0.008
ConvMF,1.840,1.144,1.355,0.321,0.237,0.098,0.139,0.008
NARRE,1.786,1.060,1.333,0.365,0.275,0.043,0.100,0.000
D_ATTN,1.847,1.073,1.356,0.354,0.265,0.022,0.120,0.000
MPCN,1.964,1.162,1.399,0.312,0.231,0.066,0.104,0.008
ALFM,1.700,1.057,1.301,0.379,0.252,0.034,0.106,0.000
TRANSNET,2.156,1.239,1.467,0.134,0.208,0.073,0.111,0.006
A3NCF,2.202,1.269,1.482,0.206,0.200,0.056,0.108,0.008
TARMF,1.856,1.081,1.359,0.380,0.286,0.094,0.134,0.000
ANR,1.793,1.091,1.337,0.355,0.259,0.062,0.095,0.000
